In [41]:
import pandas as pd
import numpy as np 
import re
import random
import copy

In [42]:
# The file that includes the slot and intent information.
df = pd.read_excel("Appliance Slot-Intent Manual.xlsx",encoding = "ISO-8859-1", sheet_name="Sheet1")
# The file that includes the types of values inside that slot. 
slot_df = pd.read_excel("Appliance Slot-Intent Manual.xlsx",encoding = "ISO-8859-1", sheet_name="Sheet3")
df['QUESTION_SHOULD_CONTAIN']= df['QUESTION_SHOULD_CONTAIN'].str.lower()
slot_df['Slot']= slot_df['Slot'].str.lower()
df['TARGET_QUESTION']= df['TARGET_QUESTION'].str.lower()
df["TARGET_QUESTION"] = df['TARGET_QUESTION'].str.replace('[\.\?\']','')

In [43]:
## Preprocessing slot_df dataframe
# Slot dataframe trimmed so that the Description is removed. 
slot_df_clean = slot_df.iloc[:,[0,2]]
# replace the Example column with an array of the values separated by a comma
slot_df_clean = slot_df_clean[slot_df_clean['Examples'].notnull()].copy()
slot_df_clean.loc[:,'Examples'] = slot_df_clean.loc[:,"Examples"].apply(lambda x: x.split(', '))
# dictionary of the slots. Key: slot value: possible vallues of the slot. 
slot_dict = dict(zip(slot_df_clean.Slot, slot_df_clean.Examples))
# Split the TARGET_QUESTION column and make it into an array of questions
df.iloc[:,1] = df.iloc[:,1].apply(lambda x: x.split('\n'))
#Temporary extract the TARGET_QUESTION column that was processed above. 
target = df.iloc[:,1]
# A Series of arrays representing the respective slots
slots = df.iloc[:,4].apply(lambda x: x.split(' '))
# The dictionary where key:Intent value:Question
target_dic = df.iloc[:,[0,1]]
# A dictionary of intents key: int value: intent
intent_dict = df.QUESTION_LABEL.to_dict()
slot_dict_boi = copy.deepcopy(slot_dict)
for key,value in slot_dict.items():
    list_slot = []
    for i in value:
        s = i.split()
        for j in range(len(s)):
            if j == 0:
                s[j] = s[j] + ":B-" + key[1:]
            else:
                s[j] = s[j] + ":I-" + key[1:]
        i = ' '.join(s)
        list_slot.append(i)
    slot_dict_boi[key] = list_slot

In [44]:
train_file_name = 'appliance_data_reverse\\appliance_utterances_train.xlsx'
test_file_name = 'appliance_data_reverse\\appliance_utterances_test2.xlsx'
valid_file_name = 'appliance_data_reverse\\appliance_utterances_valid2.xlsx'

In [45]:
df_train = pd.read_excel(train_file_name)
df_test = pd.read_excel(test_file_name)
df_valid = pd.read_excel(valid_file_name)

In [46]:
df_train.drop([114,144,146,175,173,177,194,223,230,231,281,330,331,446,451,459,517,522,529,532,585,612,613,614,615,617,626,627,626,641,652,658,659,663,677,684,685,717,732,743,742,800,806,819,826,828,827,850,851,866,872,873,875,880,886,914,928,929,949,964,988,1016,1017,1103,1013,1021,1025,1031,1048,1065,1066,1124,1126,1148,1194,1195],axis=0,inplace=True)
df_train.reset_index(drop=True,inplace=True)

In [47]:
df_train.to_excel('Application Files\\appliance_utterances_train.xlsx')

In [48]:
data_train = pd.concat([df_test, df_valid], axis=0)
data_train.reset_index(drop=True,inplace=True)

In [49]:
data_train.to_excel('appliance_data_reverse\\appliance_utterances_train_reverse.xlsx')

In [50]:
data = df_train

In [51]:
test = {"edge": [], "question_should_contain": [], "question": [], "intent":[]}
valid = {"edge": [], "question_should_contain": [], "question": [], "intent":[]}
for i in range(data.shape[0]):
    if ((i % 2 == 0)):
        test['edge'].append(data.loc[i,'edge'])
        test['question_should_contain'].append(data.loc[i,'question_should_contain'])
        test['question'].append(data.loc[i,'question'])
        test['intent'].append(data.loc[i,'intent'])
    else:
        valid['edge'].append(data.loc[i,'edge'])
        valid['question_should_contain'].append(data.loc[i,'question_should_contain'])
        valid['question'].append(data.loc[i,'question'])
        valid['intent'].append(data.loc[i,'intent'])

In [52]:
data_test = pd.DataFrame.from_dict(test)
data_valid = pd.DataFrame.from_dict(valid)

In [53]:
df_valid.reset_index(drop=True,inplace=True)
df_test.reset_index(drop=True,inplace=True)

In [54]:
data_test.to_excel('appliance_data_reverse\\appliance_utterances_test_reverse.xlsx')
data_valid.to_excel('appliance_data_reverse\\appliance_utterances_vaild_reverse.xlsx')

In [55]:
def create_data_file(f,df):
    for i in range(df.shape[0]):
        for j in range(3):
            slots = df.loc[i,'question_should_contain'].split(' ')
            x = df.loc[i,'question']
            x = re.sub(slots[0], random_slot(slots[0]), x)
            x = re.sub(slots[1], random_slot(slots[1]), x)
            x = x.lstrip()
            x = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
            x = ' '.join(x.split())
            f.write(x + " <=> " + df.loc[i,'intent'] + "\n")
            print(x + " <=> " + df.loc[i,'intent'] + "\n")
    f.close()

In [56]:
def random_slot(slot):
    random_slot = random.randint(0, len(slot_dict[slot])-1)
#     print(slot_dict[slot])
#     print(slot_dict[slot][random_slot])
    return slot_dict_boi[slot][random_slot]

In [57]:
f_train = open('C:\\Users\\l.kwon\\slot_filling_and_intent_detection_of_SLU\\data\\appliance_data_final\\train','w')
f_test = open('C:\\Users\\l.kwon\\slot_filling_and_intent_detection_of_SLU\\data\\appliance_data_final\\test','w')
f_valid = open('C:\\Users\\l.kwon\\slot_filling_and_intent_detection_of_SLU\\data\\appliance_data_final\\valid','w')

In [58]:
create_data_file(f_train,data_train)

is:O paper:B-cookware appropriate:O to:O use:O in:O the:O oven:B-appliance <=> appropriate

is:O plastic:B-cookware appropriate:O to:O use:O in:O the:O oven:B-appliance <=> appropriate

is:O wood:B-cookware appropriate:O to:O use:O in:O the:O microwave:B-appliance <=> appropriate

should:O i:O use:O the:O polyester:B-cookware cup:I-cookware in:O my:O oven:B-appliance <=> appropriate

should:O i:O use:O the:O metal:B-cookware trimes:I-cookware in:O my:O microwave:B-appliance <=> appropriate

should:O i:O use:O the:O ceramic:B-cookware in:O my:O microwave:B-appliance <=> appropriate

is:O the:O browning:B-cookware plate:I-cookware safe:O for:O use:O with:O the:O microwave:B-appliance <=> appropriate

is:O the:O glassware:B-cookware safe:O for:O use:O with:O the:O oven:B-appliance <=> appropriate

is:O the:O browning:B-cookware plate:I-cookware safe:O for:O use:O with:O the:O microwave:B-appliance <=> appropriate

is:O the:O glassware:B-cookware safe:O for:O use:O with:O the:O microwave:B

show:O me:O the:O purpose:O of:O the:O self:B-purposefeature clean:I-purposefeature on:O my:O microwave:B-appliance <=> purpose

show:O me:O the:O purpose:O of:O the:O oven:B-purposefeature clock:I-purposefeature on:O my:O microwave:B-appliance <=> purpose

how:O do:O i:O determine:O the:O purpose:O of:O display:B-purposefeature on:O the:O microwave:B-appliance <=> purpose

how:O do:O i:O determine:O the:O purpose:O of:O start:B-purposefeature set:I-purposefeature on:O the:O microwave:B-appliance <=> purpose

how:O do:O i:O determine:O the:O purpose:O of:O setting:B-purposefeature on:O the:O oven:B-appliance <=> purpose

im:O asking:O myself:O about:O i:O need:O to:O know:O what:O is:O the:O purpose:O of:O the:O dehydrate:B-purposefeature on:O the:O microwave:B-appliance <=> purpose

im:O asking:O myself:O about:O i:O need:O to:O know:O what:O is:O the:O purpose:O of:O the:O delay:B-purposefeature start:I-purposefeature on:O the:O oven:B-appliance <=> purpose

im:O asking:O myself:O ab


i:O need:O to:O know:O how:O do:O i:O clean:O the:O sugary-spills:B-cleanobject on:O the:O microwave:B-appliance <=> clean

i:O need:O to:O know:O how:O do:O i:O clean:O the:O cooktop:B-cleanobject on:O the:O microwave:B-appliance <=> clean

i:O need:O to:O know:O how:O do:O i:O clean:O the:O stainless:B-cleanobject steel:I-cleanobject on:O the:O microwave:B-appliance <=> clean

how:O should:O i:O clean:O the:O cooktop:B-cleanobject in:O the:O oven:B-appliance <=> clean

how:O should:O i:O clean:O the:O cooktop:B-cleanobject in:O the:O microwave:B-appliance <=> clean

how:O should:O i:O clean:O the:O door:B-cleanobject gasket:I-cleanobject in:O the:O microwave:B-appliance <=> clean

what:O is:O the:O best:O way:O to:O clean:O cooktop:B-cleanobject on:O the:O oven:B-appliance <=> clean

what:O is:O the:O best:O way:O to:O clean:O glass:B-cleanobject cooktop:I-cleanobject on:O the:O oven:B-appliance <=> clean

what:O is:O the:O best:O way:O to:O clean:O sugary-spills:B-cleanobject on:O 


what:O how:O can:O i:O use:O the:O light:B-usefeature buttons:I-usefeature on:O the:O oven:B-appliance <=> use

how:O can:O i:O use:O the:O sabbath:B-usefeature on:O the:O microwave:B-appliance <=> use

how:O can:O i:O use:O the:O soften:B-usefeature on:O the:O oven:B-appliance <=> use

how:O can:O i:O use:O the:O temp:B-usefeature adjust:I-usefeature on:O the:O microwave:B-appliance <=> use

is:O it:O possible:O to:O use:O 12:B-usefeature hour:I-usefeature energy:I-usefeature saving:I-usefeature in:O the:O microwave:B-appliance <=> use

is:O it:O possible:O to:O use:O vent:B-usefeature buttons:I-usefeature in:O the:O oven:B-appliance <=> use

is:O it:O possible:O to:O use:O kitchen:B-usefeature timer:I-usefeature button:I-usefeature in:O the:O oven:B-appliance <=> use

how:O do:O i:O use:O my:O dehydrate:B-usefeature in:O my:O oven:B-appliance <=> use

how:O do:O i:O use:O my:O light:B-usefeature buttons:I-usefeature in:O my:O microwave:B-appliance <=> use

how:O do:O i:O use:O my:O 

In [59]:
create_data_file(f_test,data_test)

will:O i:O be:O able:O to:O use:O plastic:B-cookware in:O the:O microwave:B-appliance <=> appropriate

will:O i:O be:O able:O to:O use:O paper:B-cookware bags:I-cookware in:O the:O oven:B-appliance <=> appropriate

will:O i:O be:O able:O to:O use:O metal:B-cookware in:O the:O microwave:B-appliance <=> appropriate

can:O i:O use:O the:O wicker:B-cookware for:O the:O microwave:B-appliance <=> appropriate

can:O i:O use:O the:O polyester:B-cookware cardboard:I-cookware for:O the:O microwave:B-appliance <=> appropriate

can:O i:O use:O the:O metal:B-cookware trimes:I-cookware for:O the:O microwave:B-appliance <=> appropriate

can:O you:O tell:O me:O if:O wicker:B-cookware is:O safe:O to:O use:O when:O cooking:O something:O in:O oven:B-appliance <=> appropriate

can:O you:O tell:O me:O if:O polyester:B-cookware cardboard:I-cookware is:O safe:O to:O use:O when:O cooking:O something:O in:O oven:B-appliance <=> appropriate

can:O you:O tell:O me:O if:O stoneware:B-cookware is:O safe:O to:O use

where:O are:O you:O door:B-removeobject microwave:B-appliance <=> remove

where:O are:O you:O rack:B-removeobject oven:B-appliance <=> remove

where:O are:O you:O rack:B-removeobject microwave:B-appliance <=> remove

yes:O please:O how:O do:O i:O remove:O door:B-removeobject from:O the:O oven:B-appliance <=> remove

yes:O please:O how:O do:O i:O remove:O stains:B-removeobject from:O the:O oven:B-appliance <=> remove

yes:O please:O how:O do:O i:O remove:O rack:B-removeobject from:O the:O oven:B-appliance <=> remove

where:O can:O i:O remove:O the:O residue:B-removeobject from:O the:O oven:B-appliance <=> remove

where:O can:O i:O remove:O the:O door:B-removeobject from:O the:O oven:B-appliance <=> remove

where:O can:O i:O remove:O the:O drawer:B-removeobject from:O the:O microwave:B-appliance <=> remove

how:O can:O i:O eliminate:O some:O residue:B-removeobject microwave:B-appliance <=> remove

how:O can:O i:O eliminate:O some:O residue:B-removeobject microwave:B-appliance <=> remove



what:O must:O i:O do:O to:O cook:O breaded:B-foodtype egg:I-foodtype plants:I-foodtype with:O the:O microwave:B-appliance d:O <=> cook

what:O must:O i:O do:O to:O cook:O breaded:B-foodtype chicken:I-foodtype breast:I-foodtype with:O the:O oven:B-appliance d:O <=> cook

how:O do:O you:O prepare:O hamburger:B-foodtype with:O the:O microwave:B-appliance <=> cook

how:O do:O you:O prepare:O frozen:B-foodtype tater:I-foodtype tots:I-foodtype with:O the:O microwave:B-appliance <=> cook

how:O do:O you:O prepare:O breaded:B-foodtype cauliflower:I-foodtype with:O the:O oven:B-appliance <=> cook

how:O do:O i:O cook:O salmon:B-foodtype steak:I-foodtype with:O the:O microwave:B-appliance <=> cook

how:O do:O i:O cook:O homemade:B-foodtype potato:I-foodtype wedges:I-foodtype with:O the:O microwave:B-appliance <=> cook

how:O do:O i:O cook:O salmon:B-foodtype steak:I-foodtype with:O the:O microwave:B-appliance <=> cook

can:O you:O show:O me:O what:O must:O i:O do:O to:O cook:O fresh:B-foodtype 

In [60]:
create_data_file(f_valid,data_valid)

is:O the:O microwave:B-appliance porcelain:B-cookware safe:O <=> appropriate

is:O the:O microwave:B-appliance wood:B-cookware safe:O <=> appropriate

is:O the:O oven:B-appliance stoneware:B-cookware safe:O <=> appropriate

will:O it:O be:O safe:O to:O use:O the:O thermometer:B-cookware on:O the:O oven:B-appliance <=> appropriate

will:O it:O be:O safe:O to:O use:O the:O browning:B-cookware plate:I-cookware on:O the:O microwave:B-appliance <=> appropriate

will:O it:O be:O safe:O to:O use:O the:O thermometer:B-cookware on:O the:O microwave:B-appliance <=> appropriate

can:O i:O safely:O use:O recycled:B-cookware paper:I-cookware on:O my:O microwave:B-appliance <=> appropriate

can:O i:O safely:O use:O metal:B-cookware trimes:I-cookware on:O my:O microwave:B-appliance <=> appropriate

can:O i:O safely:O use:O metal:B-cookware trimes:I-cookware on:O my:O oven:B-appliance <=> appropriate

is:O there:O any:O type:O of:O aluminum:B-cookware foil:I-cookware i:O should:O avoid:O using:O when:


what:O exactly:O is:O the:O purpose:O of:O setting:B-purposefeature on:O this:O microwave:B-appliance <=> purpose

want:O to:O bye:O a:O roller:B-parttype guide:I-parttype ring:I-parttype and:O microwave:B-appliance <=> buy

want:O to:O bye:O a:O door:B-parttype and:O microwave:B-appliance <=> buy

want:O to:O bye:O a:O grease:B-parttype filter:I-parttype and:O oven:B-appliance <=> buy

where:O is:O it:O possible:O to:O get:O air:B-parttype fry:I-parttype tray:I-parttype to:O get:O my:O oven:B-appliance working:O <=> buy

where:O is:O it:O possible:O to:O get:O storage:B-parttype drawer:I-parttype to:O get:O my:O microwave:B-appliance working:O <=> buy

where:O is:O it:O possible:O to:O get:O light:B-parttype to:O get:O my:O oven:B-appliance working:O <=> buy

where:O how:O i:O buy:O a:O door:B-parttype for:O my:O oven:B-appliance <=> buy

where:O how:O i:O buy:O a:O light:B-parttype for:O my:O microwave:B-appliance <=> buy

where:O how:O i:O buy:O a:O charcoal:B-parttype filter:I-par

how:O do:O i:O clean:O the:O melted-plastic:B-cleanobject on:O the:O microwave:B-appliance before:O twelve:O oclock:O <=> clean

how:O do:O i:O clean:O the:O stainless:B-cleanobject steel:I-cleanobject on:O the:O microwave:B-appliance before:O twelve:O oclock:O <=> clean

in:O which:O way:O do:O i:O clean:O cooktop:B-cleanobject on:O the:O microwave:B-appliance <=> clean

in:O which:O way:O do:O i:O clean:O sugary-spills:B-cleanobject on:O the:O oven:B-appliance <=> clean

in:O which:O way:O do:O i:O clean:O cooktop:B-cleanobject seal:I-cleanobject on:O the:O oven:B-appliance <=> clean

how:O do:O i:O clean:O the:O melted-plastic:B-cleanobject oven:B-appliance please:O please:O please:O 2:O <=> clean

how:O do:O i:O clean:O the:O sugary-spills:B-cleanobject microwave:B-appliance please:O please:O please:O 2:O <=> clean

how:O do:O i:O clean:O the:O door:B-cleanobject gasket:I-cleanobject oven:B-appliance please:O please:O please:O 2:O <=> clean

is:O it:O safe:O to:O clean:O glass:B-cl


what:O is:O the:O time:O to:O cook:O whole:B-foodtype chicken:I-foodtype in:O my:O oven:B-appliance <=> cook

what:O is:O the:O time:O to:O cook:O fresh:B-foodtype chicken:I-foodtype wings:I-foodtype in:O my:O oven:B-appliance <=> cook

how:O can:O i:O make:O it:O using:O this:O frozen:B-foodtype tater:I-foodtype tots:I-foodtype microwave:B-appliance <=> cook

how:O can:O i:O make:O it:O using:O this:O salmon:B-foodtype steak:I-foodtype oven:B-appliance <=> cook

how:O can:O i:O make:O it:O using:O this:O frozen:B-foodtype chicken:I-foodtype nugets:I-foodtype oven:B-appliance <=> cook

what:O are:O the:O steps:O to:O cook:O pizza:B-foodtype with:O the:O microwave:B-appliance <=> cook

what:O are:O the:O steps:O to:O cook:O chicken:B-foodtype pieces:I-foodtype with:O the:O oven:B-appliance <=> cook

what:O are:O the:O steps:O to:O cook:O lamb:B-foodtype chops:I-foodtype with:O the:O microwave:B-appliance <=> cook

hi:O what:O must:O i:O do:O to:O cook:O vegetables:B-foodtype with:O the


how:O to:O configure:O the:O kitchen:B-setfeature timer:I-setfeature on:O my:O microwave:B-appliance <=> set

how:O to:O configure:O the:O timed:B-setfeature cooking:I-setfeature on:O my:O oven:B-appliance <=> set

how:O to:O configure:O the:O kitchen:B-setfeature timer:I-setfeature on:O my:O oven:B-appliance <=> set

how:O do:O i:O set:O the:O convection:B-setfeature baking:I-setfeature function:O on:O my:O microwave:B-appliance easily:O <=> set

how:O do:O i:O set:O the:O timed:B-setfeature cooking:I-setfeature function:O on:O my:O microwave:B-appliance easily:O <=> set

how:O do:O i:O set:O the:O broiling:B-setfeature function:O on:O my:O oven:B-appliance easily:O <=> set

are:O there:O instructions:O to:O configure:O timed:B-setfeature cooking:I-setfeature feature:O on:O my:O oven:B-appliance <=> set

are:O there:O instructions:O to:O configure:O baking:B-setfeature feature:O on:O my:O microwave:B-appliance <=> set

are:O there:O instructions:O to:O configure:O convection:B-setfea